# `03-output_parser`

# 출력 파서 (`OutputParser`)

- LLM의 출력을 더 유용/구조화된 형태로 변환함
- 구조화: LLM의 자유 형식 출력을 구조화된 데이터로 변환
- 일관성: 항상 일관된 출력 형식 -> 후속 처리에 용이
- 유연성: 다양한 출력 형식(JSON, list, dict)으로 변환 가능

## `PydanticOutputParser`

In [ ]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

load_dotenv()

llm = ChatOpenAI(model='gpt-4.1-nano')

In [ ]:
from langchain_core.prompts import 
prompt = 